
# Challenge de Reproducibilidad: MR-DBSCAN
## Algoritmo Paralelo de Clustering por Densidad usando MapReduce

---

## EJECUCIÓN EN CLUSTER HADOOP

**Configuración requerida:**
- Jupyter Lab ejecutándose en el nodo maestro
- Spark configurado para usar YARN (cluster mode)
- Hadoop HDFS disponible para almacenamiento distribuido
- Todas las dependencias (pyspark, numpy, pandas, scikit-learn) instaladas en todos los nodos

---

## Información del Proyecto

**Paper:** MR-DBSCAN: An Efficient Parallel Density-based Clustering Algorithm using MapReduce  
**Autores:** Yaobin He, Haoyu Tan, Wuman Luo, et al. (Shenzhen Institutes of Advanced Technology)  
**Dataset:** Datos GPS reales de taxis de Shanghai (carpeta Taxi_070220)  
**Plataforma:** Hadoop + PySpark  

**Objetivo:** Reproducir el algoritmo MR-DBSCAN en 4 etapas usando MapReduce para clustering de datos espaciales a gran escala.

---

## Parámetros del Algoritmo

- **Eps (epsilon):** Radio máximo de vecindad
  - Limpieza de datos: 0.002 (elimina ruido/outliers)
  - Identificación de regiones densas: 0.0002 (clusters más específicos)

- **MinPts:** Número mínimo de puntos para formar un cluster
  - Limpieza: 1000 puntos
  - Regiones densas: 100 puntos

- **Particiones:** Grid-based partitioning
  - Strips por dimensión: 80-160 según Eps

---

## División de Tareas por Persona

| Rol | Responsabilidades |
|-----|-------------------|
| **Jose** | Etapa 1 (Preprocesamiento y Particionado) + Setup inicial |
| **Miguel** | Etapa 2 (DBSCAN Local) + Etapa 3 (Detección de Cruces) |
| **Roger** | Etapa 4 (Fusión Global) + Validación y Reportes |

---

## Fases del Algoritmo MR-DBSCAN

El algoritmo se ejecuta en 4 etapas usando MapReduce:

1. **Stage 1 - Preprocesamiento:** Análisis del dataset y particionamiento en grid
2. **Stage 2 - DBSCAN Local:** Clustering independiente en cada partición
3. **Stage 3 - Detección de Cruces:** Identificar clusters que cruzan fronteras (MC Sets)
4. **Stage 4 - Fusión Global:** Unificar clusters y relabeling global



---

# PARTE 1: SETUP INICIAL Y CARGA DE DATOS
## Responsable: Jose

### Paso 1.1: Configuración del Entorno Spark

**¿Qué hace este paso?**

Este paso inicializa una sesión de Spark (SparkSession), que es el punto de entrada principal para trabajar con DataFrames y RDDs en PySpark. La sesión establece la conexión con el cluster Hadoop y configura el nombre de la aplicación para identificarla en el Hadoop NameNode.

**Detalles técnicos:**
- `SparkSession.builder` permite crear una nueva sesión de forma fluida
- `.appName()` asigna el nombre "MR-DBSCAN" que verás en la interfaz Hadoop
- `.getOrCreate()` obtiene una sesión existente o crea una nueva

**¿Por qué es importante?**
- Necesario para distribuir datos y código a los nodos del cluster
- Permite usar funciones paralelas de Spark en todo el código



In [1]:

# Inicializar SparkSession
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

def get_spark_session(app_name='MR-DBSCAN'):
    """
    Crea una nueva sesión de Spark o retorna la existente.

    Parámetros:
        app_name (str): Nombre de la aplicación para identificar en Hadoop

    Retorna:
        SparkSession: Sesión de Spark lista para usar
    """
    # Configuración para ejecución en cluster Hadoop
    # Si estás en modo local, Spark detectará automáticamente
    # Si estás en cluster, debe estar configurado con master="yarn"
    spark = SparkSession.builder \
        .appName(app_name) \
        .config("spark.sql.shuffle.partitions", "200") \
        .config("spark.executor.memory", "2g") \
        .config("spark.executor.cores", "2") \
        .config("spark.driver.memory", "1g") \
        .getOrCreate()

    # Configurar nivel de logging para reducir mensajes
    spark.sparkContext.setLogLevel("WARN")

    return spark

# Crear sesión
spark = get_spark_session()
print(f"✓ Sesión de Spark creada exitosamente")
print(f"  - Spark Version: {spark.version}")
print(f"  - Master: {spark.sparkContext.master}")
print(f"  - Aplicación: {spark.sparkContext.appName}")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/10 18:58:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✓ Sesión de Spark creada exitosamente
  - Spark Version: 3.5.6
  - Master: local[*]
  - Aplicación: MR-DBSCAN



### 📌 Paso 1.2: Cargar y Preprocesar los Datos

**¿Qué hace este paso?**

Este paso:
1. **Verifica CSV unificado:** Busca el archivo `taxi_data_unificado.csv` (generado por `unificar_dataset.py`)
2. **Carga datos:** Si existe el CSV, lo lee directamente (más rápido). Si no existe, lo genera automáticamente.
3. **Limpieza de datos:** Elimina valores nulos e inválidos y filtra coordenadas fuera del rango válido
4. **Normalización:** Transforma las coordenadas a un rango estándar (0-1) para facilitar el particionado
5. **Estructuras de datos:** Convierte los datos a un DataFrame de Pandas (para pre-procesamiento local) o Spark (para procesamiento distribuido)

**¿Por qué es importante?**
- El MR-DBSCAN requiere datos espaciales (lat, lon) en formato limpio
- La normalización es crucial para que el grid partitioning funcione correctamente
- Los datos limpios y bien estructurados son la base del algoritmo
- Usar un CSV unificado evita tener que leer miles de archivos en cada ejecución

**Dataset:**
- Fuente: Datos GPS reales de taxis de Shanghai (carpeta Taxi_070220)
- Formato: Archivo CSV unificado `taxi_data_unificado.csv` con columnas: lon, lat
- Generación: El CSV se genera ejecutando `python unificar_dataset.py`.
- Columnas utilizadas: longitud (lon) y latitud (lat)



In [2]:

import numpy as np
import pandas as pd
import warnings
import os
import subprocess
import sys
from pathlib import Path
warnings.filterwarnings('ignore')

# Nombre del archivo CSV unificado
CSV_UNIFICADO = 'taxi_data_unificado.csv'
CARPETA_DATOS = 'Taxi_070220'

# Cargar datos desde CSV unificado
csv_path = Path(CSV_UNIFICADO)
data_folder = Path(CARPETA_DATOS)

# Verificar si el CSV unificado existe
if csv_path.exists():
    print(f" Archivo CSV unificado encontrado: {CSV_UNIFICADO}")
    print(f" Cargando datos desde CSV...")
    
    # Leer el CSV unificado
    data = pd.read_csv(csv_path)
    
    # Verificar que tiene las columnas correctas
    if 'lon' not in data.columns or 'lat' not in data.columns:
        raise ValueError(f"El archivo {CSV_UNIFICADO} no tiene las columnas 'lon' y 'lat'")
    
    print(f" Datos cargados desde CSV")
    print(f"  - Total de puntos: {len(data):,}"),

else:
    raise FileNotFoundError(
        f"El archivo {CSV_UNIFICADO} no existe. "
        f"Ejecuta primero: python unificar_dataset.py"
    )

# Limpieza y validación de datos
print(f"\n Validando y limpiando datos...")

initial_count = len(data)

# Eliminar valores nulos
data = data.dropna(subset=['lon', 'lat'])

# Filtrar coordenadas inválidas (rango válido para GPS de Shanghai)
data = data[(data['lon'] >= 120) & (data['lon'] <= 122)]
data = data[(data['lat'] >= 30) & (data['lat'] <= 32)]

if len(data) < initial_count:
    removed = initial_count - len(data)
    print(f"  - Filas eliminadas (valores inválidos): {removed:,}")
    print(f"  - Filas válidas: {len(data):,}")
else:
    print(f"  - Todas las filas son válidas: {len(data):,}")

# Normalización de coordenadas
print(f"\n Normalizando coordenadas...")

# Guardar los rangos originales para posible desnormalización
lon_min, lon_max = data['lon'].min(), data['lon'].max()
lat_min, lat_max = data['lat'].min(), data['lat'].max()

lon_range = lon_max - lon_min
lat_range = lat_max - lat_min

# Verificar que hay rango válido (evitar división por cero)
if lon_range > 0:
    data['lon_norm'] = (data['lon'] - lon_min) / lon_range
else:
    print(" Advertencia: Todos los puntos tienen la misma longitud")
    data['lon_norm'] = 0.5

if lat_range > 0:
    data['lat_norm'] = (data['lat'] - lat_min) / lat_range
else:
    print(" Advertencia: Todos los puntos tienen la misma latitud")
    data['lat_norm'] = 0.5

# Resumen final
print(f"\n" + "=" * 70)
print(f" PREPROCESAMIENTO COMPLETADO")
print(f"=" * 70)
print(f"  - Total de puntos: {len(data):,}")
print(f"  - Rango Longitud: [{lon_min:.6f}, {lon_max:.6f}]")
print(f"  - Rango Latitud: [{lat_min:.6f}, {lat_max:.6f}]")
print(f"  - Coordenadas normalizadas al rango [0, 1]")
print(f"=" * 70)

print(f"\nPrimeros 5 registros:")
print(data.head())



 Archivo CSV unificado encontrado: taxi_data_unificado.csv
 Cargando datos desde CSV...
 Datos cargados desde CSV
  - Total de puntos: 49,662

 Validando y limpiando datos...
  - Todas las filas son válidas: 49,662

 Normalizando coordenadas...

 PREPROCESAMIENTO COMPLETADO
  - Total de puntos: 49,662
  - Rango Longitud: [121.148300, 121.866100]
  - Rango Latitud: [30.886600, 31.999100]
  - Coordenadas normalizadas al rango [0, 1]

Primeros 5 registros:
        lon      lat  lon_norm  lat_norm
0  121.4666  31.2208  0.443438  0.300404
1  121.4681  31.2211  0.445528  0.300674
2  121.4695  31.2216  0.447478  0.301124
3  121.4700  31.2216  0.448175  0.301124
4  121.4695  31.2215  0.447478  0.301034



---

# PARTE 2: PARTICIONADO EN GRID
## Responsable: Jose

### Paso 2.1: División del Dominio Espacial con Grid

**¿Qué hace este paso?**

Este es el corazón del Stage 1 del MR-DBSCAN. El particionamiento en grid:

1. **Divide el espacio:** Crea una malla 2D que cubre todo el área geográfica
2. **Asigna particiones:** Cada punto GPS recibe un ID de partición según su ubicación en la malla
3. **Equilibra carga:** Intenta que cada partición tenga aproximadamente la misma cantidad de datos

**Conceptos clave:**
- **n_strips:** Número de divisiones por dimensión (80-160 según el paper)
  - Más strips = particiones más pequeñas (mejor balance pero más réplicas en bordes)
  - Menos strips = particiones más grandes (menos comunicación pero desbalance)
- **lon_bin / lat_bin:** Índice de la celda en cada dimensión
- **partition_id:** Identificador único de la partición (combinación de lon_bin y lat_bin)

**¿Por qué es importante?**
- MapReduce distribuye datos a diferentes nodos según la partición
- El balance de carga afecta directamente el rendimiento paralelo
- Un particionamiento pobre causa "data skew" (algunos nodos reciben más datos)

**Ejemplo visual:**
```
Si n_strips=3, espacio 0-1 se divide en:
[0-0.33, 0.33-0.67, 0.67-1.0]

Un punto en (0.5, 0.8) recibe:
lon_bin = 1 (está en [0.33-0.67])
lat_bin = 2 (está en [0.67-1.0])
partition_id = "1_2"
```



In [3]:

# Definir parámetros de particionamiento
# Guardar n_strips como variable global para usar en detección de fronteras
n_strips = 120  # Óptimo según el paper para Eps grande (0.002)

# Crear bins para dividir el espacio normalizado [0, 1]
# np.linspace crea n_strips+1 límites, generando n_strips intervalos
lon_bins = np.linspace(0, 1, n_strips + 1)
lat_bins = np.linspace(0, 1, n_strips + 1)

# np.digitize con right=False para manejar correctamente los bordes
# Los puntos exactamente en el límite superior deben ir al bin anterior
# Usamos np.searchsorted que es más preciso para este caso
data['lon_bin'] = np.searchsorted(lon_bins, data['lon_norm'], side='right') - 1
data['lat_bin'] = np.searchsorted(lat_bins, data['lat_norm'], side='right') - 1

# Asegurar que los bins estén en el rango válido [0, n_strips-1]
# Los puntos en el límite superior (1.0) pueden dar n_strips, los ajustamos
data['lon_bin'] = np.clip(data['lon_bin'], 0, n_strips - 1)
data['lat_bin'] = np.clip(data['lat_bin'], 0, n_strips - 1)

# Crear identificador único de partición
data['partition_id'] = data['lon_bin'].astype(str) + '_' + data['lat_bin'].astype(str)

# Información del particionado
n_partitions = len(data['partition_id'].unique())
print(f" Particionamiento completado")
print(f"  - Número de strips por dimensión: {n_strips}")
print(f"  - Total de particiones únicas con datos: {n_partitions}")
print(f"  - Total de particiones posibles en grid: {n_strips * n_strips}")
print(f"  - Puntos por partición (promedio): {len(data) / n_partitions:.1f}")

# Mostrar distribución de datos entre particiones
partition_counts = data['partition_id'].value_counts()
print(f"\nDistribución de datos:")
print(f"  - Partición con más datos: {partition_counts.max()} puntos")
print(f"  - Partición con menos datos: {partition_counts.min()} puntos")
print(f"  - Desviación estándar: {partition_counts.std():.1f}")

# Guardar información de bins para usar en detección de fronteras
# Esto se usará después para determinar los límites reales de cada celda
print(f"\n Información de bins guardada para detección de fronteras")
print(f"  - Límites longitud: [{lon_bins[0]:.6f}, {lon_bins[-1]:.6f}]")
print(f"  - Límites latitud: [{lat_bins[0]:.6f}, {lat_bins[-1]:.6f}]")


 Particionamiento completado
  - Número de strips por dimensión: 120
  - Total de particiones únicas con datos: 1873
  - Total de particiones posibles en grid: 14400
  - Puntos por partición (promedio): 26.5

Distribución de datos:
  - Partición con más datos: 1318 puntos
  - Partición con menos datos: 1 puntos
  - Desviación estándar: 69.2

 Información de bins guardada para detección de fronteras
  - Límites longitud: [0.000000, 1.000000]
  - Límites latitud: [0.000000, 1.000000]



### Paso 2.2: Preparación de Datos para MapReduce

**¿Qué hace este paso?**

Prepara los datos para ser procesados en paralelo en Hadoop:

1. **Agrupa por partición:** Organiza todos los puntos que pertenecen a la misma partición
2. **Convierte a Spark DataFrame:** Transforma los datos de Pandas a Spark para procesamiento distribuido
3. **Repartición:** Distribuye las particiones entre los nodos del cluster Hadoop

**¿Por qué es importante?**
- MapReduce requiere datos en formato distribuido
- Spark DataFrame permite operaciones paralelas automáticas
- La repartición asegura que cada nodo reciba trabajo similar



In [4]:
# Preparación de datos para MapReduce
# Opción 1: Mantener en Pandas para procesamiento local
partition_groups = dict(list(data.groupby('partition_id')))
print(f"✓ Datos agrupados por partición (Pandas)")
print(f"  - Número de grupos: {len(partition_groups)}")
print(f"  - Ejemplo de keys: {list(partition_groups.keys())[:5]}")

# Opción 2: Convertir a Spark DataFrame para procesamiento distribuido
# Usamos 200 para mejor balance de carga y evitar "data skew"
n_spark_partitions = 200  # Número recomendado de particiones Spark

try:
    # Convertir DataFrame de Pandas a Spark
    # Esto permite procesamiento distribuido en el cluster Hadoop
    spark_df = spark.createDataFrame(data)
    
    # Reparticionar por partition_id con número óptimo de particiones
    # Esto distribuye mejor la carga entre los nodos del cluster y evita warnings
    # Cada partición del grid puede estar en diferentes nodos para paralelismo real
    spark_df = spark_df.repartition(n_spark_partitions, 'partition_id')
    
    # Cachear el DataFrame si se va a usar múltiples veces
    # Para este caso, solo lo usamos para mostrar información, no lo cacheamos
    
    print(f"\n Spark DataFrame creado y optimizado para cluster Hadoop")
    print(f"  - Número de particiones Spark: {spark_df.rdd.getNumPartitions()}")
    print(f"  - Total de registros: {spark_df.count():,}")
    print(f"  - Particiones de datos únicas (grids): {spark_df.select('partition_id').distinct().count()}")
    print(f"\nEstructura de datos:")
    spark_df.printSchema()
    
except Exception as e:
    print(f"\n Advertencia: Error al crear Spark DataFrame: {e}")
    print("  Continuando con procesamiento local usando Pandas...")
    print("  Para ejecución en cluster Hadoop, verifica la configuración de Spark")
    spark_df = None


✓ Datos agrupados por partición (Pandas)
  - Número de grupos: 1873
  - Ejemplo de keys: ['0_12', '0_14', '0_45', '100_29', '100_31']

 Spark DataFrame creado y optimizado para cluster Hadoop


25/11/10 18:58:44 WARN TaskSetManager: Stage 0 contains a task of very large size (2482 KiB). The maximum recommended task size is 1000 KiB.
[Stage 0:>                                                          (0 + 1) / 1]

  - Número de particiones Spark: 200


25/11/10 18:58:49 WARN TaskSetManager: Stage 1 contains a task of very large size (2482 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

  - Total de registros: 49,662


25/11/10 18:58:57 WARN TaskSetManager: Stage 7 contains a task of very large size (2482 KiB). The maximum recommended task size is 1000 KiB.
[Stage 9:====================================================>  (191 + 1) / 200]

  - Particiones de datos únicas (grids): 1873

Estructura de datos:
root
 |-- lon: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon_norm: double (nullable = true)
 |-- lat_norm: double (nullable = true)
 |-- lon_bin: long (nullable = true)
 |-- lat_bin: long (nullable = true)
 |-- partition_id: string (nullable = true)




---

# PARTE 3: DBSCAN LOCAL Y GENERACIÓN DE MC SETS
## Responsable: Miguel

### Paso 3.1: DBSCAN Local en Cada Partición

**¿Qué hace este paso? (Stage 2 del paper)**

Este es el corazón del clustering. Para cada partición:

1. **Ejecuta DBSCAN localmente:** Aplica el algoritmo clásico DBSCAN a los puntos de la partición
2. **Clasifica puntos:** Etiqueta cada punto como:
   - **Core Point (C):** Tiene ≥ MinPts vecinos dentro de Eps
   - **Border Point (B):** Tiene < MinPts vecinos pero es vecino de un Core Point
   - **Noise (N):** No cumple ninguna condición (outlier)
3. **Genera clusters locales:** Crea cluster IDs locales por partición

**Parámetros importantes:**
- **Eps = 0.002:** Radio de búsqueda (distancia máxima a vecinos)
- **MinPts = 1000:** Mínimo número de puntos para ser Core Point

**Algoritmo DBSCAN en pseudocódigo:**
```
Para cada punto p no visitado:
  1. Si p tiene < MinPts vecinos → marcar como Noise
  2. Si p tiene ≥ MinPts vecinos → marcar como Core Point
  3. Expandir: recursivamente procesar todos sus vecinos
  4. Crear cluster cuando se termina la expansión
```

**¿Por qué es importante?**
- Cada nodo ejecuta esta operación independientemente (paralelismo real)
- Los resultados locales se combinan después en el Stage 4
- Reduce significativamente la complejidad computacional general

**Complejidad:**
- Tiempo: O(n * log n) con indexación espacial, O(n²) sin indexación
- Espacio: O(n) para almacenar el dataset



In [5]:

from sklearn.cluster import DBSCAN
from sklearn.neighbors import KDTree
import scipy.spatial.distance as spatial_distance

# Definir parámetros DBSCAN
EPS = 0.002  # Radio de búsqueda en coordenadas normalizadas
MIN_SAMPLES = 1000  # Mínimo número de puntos para ser Core Point

print(f"Parámetros de DBSCAN:")
print(f"  - Eps: {EPS}")
print(f"  - MinPts: {MIN_SAMPLES}")

def local_dbscan_partition(partition_data, eps=EPS, min_samples=MIN_SAMPLES):
    """
    Ejecuta DBSCAN en una partición individual.

    Parámetros:
        partition_data: DataFrame con puntos de una partición
        eps: Radio de búsqueda
        min_samples: Mínimo de puntos para Core Point

    Retorna:
        array: Etiquetas de cluster locales para cada punto (-1 = noise)
    """
    coords = partition_data[['lon_norm', 'lat_norm']].values

    # Si la partición tiene pocos puntos, todos son noise
    if len(coords) < min_samples:
        return np.array([-1] * len(coords))

    # Ejecutar DBSCAN
    db = DBSCAN(eps=eps, min_samples=min_samples, metric='euclidean', n_jobs=-1)
    labels = db.fit_predict(coords)

    return labels

# Aplicar DBSCAN a cada partición
print(f"\n Ejecutando DBSCAN local en {n_partitions} particiones...")

data['local_cluster'] = -1  # Inicializar

for partition_id, group_idx in data.groupby('partition_id').groups.items():
    group = data.loc[group_idx]
    labels = local_dbscan_partition(group)
    data.loc[group_idx, 'local_cluster'] = labels

print(f" DBSCAN local completado")
print(f"  - Puntos en clusters: {(data['local_cluster'] != -1).sum():,}")
print(f"  - Puntos noise/outliers: {(data['local_cluster'] == -1).sum():,}")

# Calcular número de clusters locales de forma segura
clusters_local = data[data['local_cluster'] != -1]['local_cluster']
if len(clusters_local) > 0:
    n_local_clusters = clusters_local.nunique()
    max_cluster_id = clusters_local.max()
    print(f"  - Total de clusters locales únicos: {n_local_clusters}")
    print(f"  - ID máximo de cluster local: {max_cluster_id}")
else:
    print(f"  - Total de clusters locales: 0 (todos los puntos son noise)")
    n_local_clusters = 0

# Mostrar estadísticas por partición
print(f"\nEstadísticas por partición:")
try:
    # Calcular puntos en clusters por partición
    stats = data.groupby('partition_id').agg({
        'local_cluster': lambda x: (x != -1).sum(),  # Puntos en clusters
    })
    stats = stats.rename(columns={'local_cluster': 'clusters'})
    # Ahora agregar total_points y noise
    stats['total_points'] = data.groupby('partition_id').size()
    stats['noise'] = stats['total_points'] - stats['clusters']
    print(stats.head(10))
except Exception as e:
    print(f" Error al calcular estadísticas: {e}")
    print("  Continuando con el siguiente paso...")


Parámetros de DBSCAN:
  - Eps: 0.002
  - MinPts: 1000

 Ejecutando DBSCAN local en 1873 particiones...
 DBSCAN local completado
  - Puntos en clusters: 2,436
  - Puntos noise/outliers: 47,226
  - Total de clusters locales únicos: 1
  - ID máximo de cluster local: 0

Estadísticas por partición:
              clusters  total_points  noise
partition_id                               
0_12                 0             1      1
0_14                 0             2      2
0_45                 0             2      2
100_29               0            11     11
100_31               0             8      8
100_32               0             7      7
100_33               0             8      8
100_34               0             2      2
101_28               0            13     13
101_31               0            13     13



### Paso 3.2: Identificación de Puntos Frontera (MC Sets)

**¿Qué hace este paso? (Preparación para Stage 3)**

Identifica puntos especiales que **cruzan fronteras entre particiones**:

1. **Puntos frontera:** Puntos cuyo vecindario Eps-distance cruza hacia otra partición
2. **MC Sets (Merge Candidate Sets):** Conjuntos de puntos que potencialmente deben fusionarse
3. **Clasificación:** Marca los puntos que necesitarán procesamiento especial

**¿Por qué es importante?**
- DBSCAN local NO ve puntos en otras particiones
- Dos clusters en particiones vecinas podrían ser el MISMO cluster globalmente
- Necesitamos identificar estos casos ANTES de hacer la fusión

**Concepto clave - El problema de las fronteras:**

```
Partición A        |  Partición B
    ●●●●●●●●●●●●●●|●●●●●●●●●●●●●
    ●●●●●●●●●●●●●●|●●●●●●●●●●●●●

Local en A: cluster 1
Local en B: cluster 1
Pero podrían ser el MISMO cluster si un punto de A está cerca de B

→ Los puntos cerca de la frontera deben compararse entre particiones
```



In [6]:

# Identificar puntos frontera usando límites reales de la celda del grid
def identify_border_points(partition_data, partition_id, lon_bins, lat_bins, eps=EPS):
    """
    Identifica puntos de una partición que están cerca de las fronteras de su celda del grid.

    Parámetros:
        partition_data: DataFrame con puntos de una partición
        partition_id: ID de la partición (formato "lon_bin_lat_bin")
        lon_bins: Array con límites de bins para longitud
        lat_bins: Array con límites de bins para latitud
        eps: Radio de búsqueda Eps

    """
    coords = partition_data[['lon_norm', 'lat_norm']].values

    if len(coords) == 0:
        return np.array([], dtype=bool)

    # Obtener los límites reales de la celda del grid
    # Parsear partition_id para obtener los índices de bin
    try:
        lon_bin_idx, lat_bin_idx = map(int, partition_id.split('_'))
    except:
        # Si falla el parsing, usar método alternativo (límites de datos)
        lon_min, lon_max = coords[:, 0].min(), coords[:, 0].max()
        lat_min, lat_max = coords[:, 1].min(), coords[:, 1].max()
    else:
        # Usar los límites reales de la celda del grid
        lon_min = lon_bins[lon_bin_idx]
        lon_max = lon_bins[lon_bin_idx + 1] if lon_bin_idx + 1 < len(lon_bins) else lon_bins[-1]
        lat_min = lat_bins[lat_bin_idx]
        lat_max = lat_bins[lat_bin_idx + 1] if lat_bin_idx + 1 < len(lat_bins) else lat_bins[-1]

    # Comprobar si está dentro de Eps de cualquier borde de la celda
    # Esto asegura que detectamos puntos que pueden conectarse con particiones vecinas
    is_border = (
        (coords[:, 0] <= lon_min + eps) |  # Cerca borde oeste (o dentro de eps)
        (coords[:, 0] >= lon_max - eps) |  # Cerca borde este (o dentro de eps)
        (coords[:, 1] <= lat_min + eps) |  # Cerca borde sur (o dentro de eps)
        (coords[:, 1] >= lat_max - eps)    # Cerca borde norte (o dentro de eps)
    )

    return is_border

# Aplicar detección de fronteras usando límites de celdas del grid
print(f" Identificando puntos frontera usando límites de celdas del grid...")
data['is_border'] = False

# Asegurar que lon_bins y lat_bins estén disponibles (definidos en paso anterior)
# Si no están definidos, recrearlos
if 'lon_bins' not in globals() or 'lat_bins' not in globals():
    lon_bins = np.linspace(0, 1, n_strips + 1)
    lat_bins = np.linspace(0, 1, n_strips + 1)

for partition_id, group_idx in data.groupby('partition_id').groups.items():
    group = data.loc[group_idx]
    border_flags = identify_border_points(group, partition_id, lon_bins, lat_bins, eps=EPS)
    data.loc[group_idx, 'is_border'] = border_flags

# Mostrar resultados
n_border = data['is_border'].sum()
pct_border = 100 * n_border / len(data)

print(f" Puntos frontera identificados")
print(f"  - Total puntos frontera: {n_border:,} ({pct_border:.1f}%)")
print(f"  - Puntos internos: {(~data['is_border']).sum():,}")

# Crear MC Sets (Merge Candidate Sets)
# Estos son pares de clusters que potencialmente deben fusionarse
mc_sets = []

for partition_id, group_idx in data.groupby('partition_id').groups.items():
    group = data.loc[group_idx]
    border_points = group[group['is_border']]

    if len(border_points) > 0:
        for idx, row in border_points.iterrows():
            if row['local_cluster'] != -1:  # No contar noise points
                mc_sets.append({
                    'partition_id': partition_id,
                    'point_id': idx,
                    'cluster_id': row['local_cluster'],
                    'lon': row['lon_norm'],
                    'lat': row['lat_norm']
                })

mc_df = pd.DataFrame(mc_sets)
print(f"\n MC Sets (Merge Candidates) creados")
print(f"  - Total de candidatos para fusión: {len(mc_df):,}")
if len(mc_df) > 0:
    print(f"  - Clusters únicos en MC: {mc_df['cluster_id'].nunique()}")
    print(f"  - Particiones con candidatos: {mc_df['partition_id'].nunique()}")


 Identificando puntos frontera usando límites de celdas del grid...
 Puntos frontera identificados
  - Total puntos frontera: 34,814 (70.1%)
  - Puntos internos: 14,848

 MC Sets (Merge Candidates) creados
  - Total de candidatos para fusión: 265
  - Clusters únicos en MC: 1
  - Particiones con candidatos: 1



---

# PARTE 4: FUSIÓN DE CLUSTERS Y GLOBAL MERGE
## Responsable: Roger

### Paso 4.1: Mapeo y Generación de IDs Globales

**¿Qué hace este paso? (Stage 3 y 4.1 del paper)**

Crea un **mapa de equivalencia** entre IDs locales y globales:

1. **Busca pares para fusionar:** Usa MC Sets para encontrar clusters que deben unirse
2. **Crea tabla de mapeo:** Tabla que dice "cluster local (A, 5) → cluster global 102"
3. **Union-Find:** Usa algoritmo eficiente para manejar transitividad
   - Si A-B deben fusionarse y B-C deben fusionarse → A-B-C son el mismo cluster

**¿Por qué es importante?**
- Un cluster puede atravesar múltiples particiones
- Los clusters deben tener IDs únicos globales, no locales
- La transitividad es clave: si varias particiones conectan clusters, todos deben unificarse

**Ejemplo:**
```
Partición 0: cluster 1 (contiene puntos x, y)
Partición 1: cluster 3 (contiene puntos y, z)
Puntos y están en ambas particiones y cercanos en espacio

→ Mapeo: (0, 1) → 100, (1, 3) → 100  (mismo ID global)
```



In [7]:

class UnionFind:
    """
    Estructura de datos Union-Find (Disjoint Set Union) para gestionar
    equivalencias entre clusters.
    """
    def __init__(self):
        self.parent = {}
        self.rank = {}

    def find(self, x):
        """Encuentra el representante del conjunto de x (compresión de caminos)"""
        if x not in self.parent:
            self.parent[x] = x
            self.rank[x] = 0

        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])  # Compresión de caminos

        return self.parent[x]

    def union(self, x, y):
        """Une los conjuntos que contienen x e y (unión por rango)"""
        root_x = self.find(x)
        root_y = self.find(y)

        if root_x != root_y:
            # Unión por rango (árbol más pequeño se cuelga del más grande)
            if self.rank[root_x] < self.rank[root_y]:
                self.parent[root_x] = root_y
            elif self.rank[root_x] > self.rank[root_y]:
                self.parent[root_y] = root_x
            else:
                self.parent[root_y] = root_x
                self.rank[root_x] += 1

# Crear estructura Union-Find y manejar caso de MC Sets vacío
uf = UnionFind()

# Verificar que mc_df no esté vacío antes de procesar
print(f" Buscando clusters para fusionar en fronteras...")

merge_count = 0
if len(mc_df) == 0:
    print("  No hay puntos frontera (MC Sets vacío). No se pueden fusionar clusters entre particiones.")
    print("  Esto puede ocurrir si los clusters no cruzan fronteras de particiones.")
else:
    # Dos puntos deben estar a distancia <= EPS para estar en el mismo cluster
    dist_threshold = EPS
    
    # Solo comparar cada par una vez (idx_a < idx_b)
    n_mc_points = len(mc_df)
    print(f"  - Procesando {n_mc_points:,} puntos frontera...")
    
    for idx_a in range(n_mc_points):
        row_a = mc_df.iloc[idx_a]
        partition_a = row_a['partition_id']
        
        # Solo comparar con puntos de otras particiones
        for idx_b in range(idx_a + 1, n_mc_points):
            row_b = mc_df.iloc[idx_b]
            partition_b = row_b['partition_id']
            
            # No comparar dentro de la misma partición
            if partition_a == partition_b:
                continue
            
            # Calcular distancia euclidiana entre puntos normalizados
            dist = np.sqrt((row_a['lon'] - row_b['lon'])**2 +
                           (row_a['lat'] - row_b['lat'])**2)
            
            # Si distancia <= EPS, los clusters deben fusionarse
            if dist <= dist_threshold:
                # Estos puntos están cercanos: sus clusters deben fusionarse
                # Asegurar que partition_id sea string para consistencia
                key_a = (str(row_a['partition_id']), row_a['cluster_id'])
                key_b = (str(row_b['partition_id']), row_b['cluster_id'])
                
                # Solo contar si realmente se unen
                if uf.find(key_a) != uf.find(key_b):
                    uf.union(key_a, key_b)
                    merge_count += 1
    
    print(f" Fusiones completadas: {merge_count} pares de clusters unidos")
    if merge_count == 0:
        print(" No se encontraron clusters para fusionar entre particiones.")

# Construir tabla de mapeo de forma más robusta
print(f"\n Creando tabla de mapeo global...")

mapping_table = {}
global_id = 0
seen_roots = {}

# Asignar IDs globales basados en componentes conectados
# Recorrer todas las particiones y sus clusters locales
for partition_id in data['partition_id'].unique():
    partition_data = data[data['partition_id'] == partition_id]
    
    # Obtener clusters locales únicos (excluyendo noise)
    local_clusters = partition_data[partition_data['local_cluster'] != -1]['local_cluster'].unique()
    
    for local_cluster_id in local_clusters:
        # Asegurar que partition_id sea string para consistencia con relabeling
        key = (str(partition_id), local_cluster_id)
        
        # Si el cluster no está en Union-Find (no se fusionó)
        # crear una entrada para él
        root = uf.find(key)
        
        if root not in seen_roots:
            seen_roots[root] = global_id
            global_id += 1
        
        mapping_table[key] = seen_roots[root]

print(f" Tabla de mapeo creada")
print(f"  - Entradas en tabla: {len(mapping_table)}")
print(f"  - Total clusters globales: {global_id}")

# Mostrar algunas entradas del mapeo solo si hay datos
if len(mapping_table) > 0:
    print(f"\nEjemplos del mapeo (primeros 5):")
    for i, (key, gid) in enumerate(list(mapping_table.items())[:5]):
        print(f"  {key} → {gid}")
else:
    print(f"\n No hay clusters para mapear (todos los puntos son noise)")


 Buscando clusters para fusionar en fronteras...
  - Procesando 265 puntos frontera...
 Fusiones completadas: 0 pares de clusters unidos
 No se encontraron clusters para fusionar entre particiones.

 Creando tabla de mapeo global...
 Tabla de mapeo creada
  - Entradas en tabla: 2
  - Total clusters globales: 2

Ejemplos del mapeo (primeros 5):
  ('57_53', np.int64(0)) → 0
  ('74_42', np.int64(0)) → 1



### Paso 4.2: Relabeling Global (Reemplazar IDs Locales por Globales)

**¿Qué hace este paso? (Stage 4.2 del paper)**

Recorre todos los puntos y reemplaza sus cluster IDs locales por los globales:

1. **Mapeo directo:** Para cada punto, busca su (partition_id, local_cluster) en la tabla
2. **Asignación:** Reemplaza el local_cluster con el global_cluster
3. **Resultado final:** Todos los puntos tienen un cluster global único

**¿Por qué es importante?**
- Después de este paso tenemos el resultado FINAL del clustering
- Dos puntos en particiones diferentes que deberían estar en el mismo cluster ahora lo están
- El algoritmo MR-DBSCAN está completo



In [8]:

def relabel_to_global(row, mapping_table):
    """
    Convierte un cluster ID local a global usando la tabla de mapeo.
    
    Manejo seguro de casos donde el cluster no está en mapping_table

    Parámetros:
        row: Fila del DataFrame con columnas partition_id y local_cluster
        mapping_table: Dict con mapeo (partition_id, local_cluster) → global_id
    """
    # Si es noise, retornar -1 inmediatamente
    if row['local_cluster'] == -1:
        return -1

    # Verificar que mapping_table no esté vacío
    if not mapping_table:
        print(" Advertencia: mapping_table está vacío. Todos los puntos serán marcados como noise.")
        return -1

    # Construir key y buscar en mapping_table
    # Usar str() para asegurar que partition_id sea string (puede ser int)
    key = (str(row['partition_id']), row['local_cluster'])

    # Si no se encuentra el key, retornar -1 (noise)
    # Esto puede ocurrir si hay un error en la construcción del mapeo
    global_id = mapping_table.get(key, -1)
    
    if global_id == -1 and row['local_cluster'] != -1:
        pass
    
    return global_id

# Aplicar relabeling
print(f" Aplicando relabeling global a todos los puntos...")
data['global_cluster'] = data.apply(
    lambda row: relabel_to_global(row, mapping_table),
    axis=1
)

print(f" Relabeling completado")

# Estadísticas finales con manejo de casos edge
n_global_clusters_points = (data['global_cluster'] != -1).sum()
n_noise_global = (data['global_cluster'] == -1).sum()

print(f"\n═══ RESULTADOS FINALES DEL MR-DBSCAN ═══")
print(f"  - Total de puntos: {len(data):,}")
print(f"  - Puntos en clusters: {n_global_clusters_points:,}")
print(f"  - Puntos noise (outliers): {n_noise_global:,}")

# Calcular número de clusters globales de forma segura
clusters_global = data[data['global_cluster'] != -1]['global_cluster']
if len(clusters_global) > 0:
    n_global_clusters_unique = clusters_global.nunique()
    max_global_cluster_id = clusters_global.max()
    print(f"  - Número total de clusters globales únicos: {n_global_clusters_unique}")
    print(f"  - ID máximo de cluster global: {max_global_cluster_id}")
else:
    n_global_clusters_unique = 0
    print(f"  - Número total de clusters globales: 0 (todos los puntos son noise)")

if len(data) > 0:
    purity = 100 * n_global_clusters_points / len(data)
    print(f"  - Pureza del clustering: {purity:.1f}%")

# Distribución de clusters con manejo de casos vacíos
if len(clusters_global) > 0:
    cluster_sizes = clusters_global.value_counts()
    print(f"\nDistribución de clusters globales:")
    print(f"  - Cluster más grande: {cluster_sizes.max():,} puntos")
    print(f"  - Cluster más pequeño: {cluster_sizes.min()} puntos")
    print(f"  - Tamaño promedio: {cluster_sizes.mean():.1f} puntos")
    print(f"  - Mediana: {cluster_sizes.median():.1f} puntos")
    
    print(f"\nPrimeros 10 clusters por tamaño:")
    for cluster_id, size in cluster_sizes.head(10).items():
        print(f"  Cluster {int(cluster_id)}: {size:,} puntos")
else:
    print(f"\n No hay clusters para mostrar (todos los puntos son noise)")
    cluster_sizes = pd.Series(dtype=int)  # Crear Series vacío para evitar errores después


 Aplicando relabeling global a todos los puntos...
 Relabeling completado

═══ RESULTADOS FINALES DEL MR-DBSCAN ═══
  - Total de puntos: 49,662
  - Puntos en clusters: 2,436
  - Puntos noise (outliers): 47,226
  - Número total de clusters globales únicos: 2
  - ID máximo de cluster global: 1
  - Pureza del clustering: 4.9%

Distribución de clusters globales:
  - Cluster más grande: 1,281 puntos
  - Cluster más pequeño: 1155 puntos
  - Tamaño promedio: 1218.0 puntos
  - Mediana: 1218.0 puntos

Primeros 10 clusters por tamaño:
  Cluster 1: 1,281 puntos
  Cluster 0: 1,155 puntos



---

# PARTE 5: VALIDACIÓN Y ALMACENAMIENTO DE RESULTADOS

### Paso 5.1: Guardar Resultados

**¿Qué hace este paso?**

Almacena los resultados finales en archivos para:
- Documentación y auditoría
- Visualización posterior
- Comparación con resultados esperados



In [9]:

# Guardar resultados completos a CSV
output_file = 'mr_dbscan_results.csv'
data.to_csv(output_file, index=False)
print(f" Resultados guardados en: {output_file}")
print(f"  - Columnas: {', '.join(data.columns.tolist())}")

# Guardar resumen de clustering
summary_data = data[['lon', 'lat', 'global_cluster']].copy()
summary_file = 'clustering_summary.csv'
summary_data.to_csv(summary_file, index=False)
print(f" Resumen guardado en: {summary_file}")

# Crear reporte de estadísticas con manejo seguro de variables
# Verificar que todas las variables estén definidas antes de usarlas
try:
    # Obtener estadísticas de clusters locales
    clusters_local_unique = data[data['local_cluster'] != -1]['local_cluster'].nunique() if len(data[data['local_cluster'] != -1]) > 0 else 0
    n_local_clustered = (data['local_cluster'] != -1).sum()
    n_local_noise = (data['local_cluster'] == -1).sum()
    
    # Obtener estadísticas de clusters globales
    clusters_global_unique = data[data['global_cluster'] != -1]['global_cluster'].nunique() if len(data[data['global_cluster'] != -1]) > 0 else 0
    n_global_clustered = (data['global_cluster'] != -1).sum()
    n_global_noise = (data['global_cluster'] == -1).sum()
    
    # Estadísticas de distribución de clusters
    if len(data[data['global_cluster'] != -1]) > 0:
        cluster_sizes_report = data[data['global_cluster'] != -1]['global_cluster'].value_counts()
        max_cluster_size = cluster_sizes_report.max()
        min_cluster_size = cluster_sizes_report.min()
        mean_cluster_size = cluster_sizes_report.mean()
        std_cluster_size = cluster_sizes_report.std()
    else:
        max_cluster_size = 0
        min_cluster_size = 0
        mean_cluster_size = 0.0
        std_cluster_size = 0.0
    
    # MC Sets
    mc_sets_count = len(mc_df) if 'mc_df' in globals() and len(mc_df) > 0 else 0
    
    # Pureza
    purity_pct = (100 * n_global_clustered / len(data)) if len(data) > 0 else 0.0
    
    # Promedio de puntos por partición
    avg_points_per_partition = (len(data) / n_partitions) if n_partitions > 0 else 0.0
    
except Exception as e:
    print(f" Error al preparar estadísticas para el reporte: {e}")
    # Valores por defecto en caso de error
    clusters_local_unique = 0
    n_local_clustered = 0
    n_local_noise = len(data)
    clusters_global_unique = 0
    n_global_clustered = 0
    n_global_noise = len(data)
    max_cluster_size = 0
    min_cluster_size = 0
    mean_cluster_size = 0.0
    std_cluster_size = 0.0
    mc_sets_count = 0
    purity_pct = 0.0
    avg_points_per_partition = 0.0

report_file = 'mr_dbscan_report.txt'

# Crear reporte
report = f"""
═══════════════════════════════════════════════════════════
          REPORTE FINAL DEL MR-DBSCAN CHALLENGE
═══════════════════════════════════════════════════════════

CONFIGURACIÓN DEL ALGORITMO:
  - Eps (epsilon): {EPS}
  - MinPts (mínimo de puntos): {MIN_SAMPLES}
  - Número de particiones (strips): {n_strips}

DATOS DE ENTRADA:
  - Total de puntos: {len(data):,}
  - Rango Longitud: [{data['lon'].min():.6f}, {data['lon'].max():.6f}]
  - Rango Latitud: [{data['lat'].min():.6f}, {data['lat'].max():.6f}]

STAGE 1 - PARTICIONAMIENTO:
  - Número total de particiones con datos: {n_partitions}
  - Puntos por partición (promedio): {avg_points_per_partition:.1f}

STAGE 2 - DBSCAN LOCAL:
  - Clusters locales identificados: {clusters_local_unique}
  - Puntos en clusters locales: {n_local_clustered:,}
  - Puntos noise/outliers locales: {n_local_noise:,}

STAGE 3 - DETECCIÓN DE CRUCES:
  - Puntos frontera identificados: {data['is_border'].sum():,}
  - MC Sets (candidatos para fusión): {mc_sets_count:,}
  - Pares de clusters fusionados: {merge_count if 'merge_count' in globals() else 0}

STAGE 4 - FUSIÓN GLOBAL:
  - Clusters globales finales: {clusters_global_unique}
  - Puntos en clusters globales: {n_global_clustered:,}
  - Puntos noise finales: {n_global_noise:,}

CALIDAD DE CLUSTERING:
  - Pureza (% puntos en clusters): {purity_pct:.1f}%
  - Tamaño máximo de cluster: {max_cluster_size:,} puntos
  - Tamaño mínimo de cluster: {min_cluster_size} puntos
  - Tamaño promedio: {mean_cluster_size:.1f} puntos
  - Desviación estándar: {std_cluster_size:.1f}

ARCHIVOS GENERADOS:
  - {output_file}
  - {summary_file}
  - {report_file}

═══════════════════════════════════════════════════════════
"""
with open(report_file, 'w') as f:
    f.write(report)

print(f" Reporte guardado en: {report_file}")
print(report)


 Resultados guardados en: mr_dbscan_results.csv
  - Columnas: lon, lat, lon_norm, lat_norm, lon_bin, lat_bin, partition_id, local_cluster, is_border, global_cluster
 Resumen guardado en: clustering_summary.csv
 Reporte guardado en: mr_dbscan_report.txt

═══════════════════════════════════════════════════════════
          REPORTE FINAL DEL MR-DBSCAN CHALLENGE
═══════════════════════════════════════════════════════════

CONFIGURACIÓN DEL ALGORITMO:
  - Eps (epsilon): 0.002
  - MinPts (mínimo de puntos): 1000
  - Número de particiones (strips): 120

DATOS DE ENTRADA:
  - Total de puntos: 49,662
  - Rango Longitud: [121.148300, 121.866100]
  - Rango Latitud: [30.886600, 31.999100]

STAGE 1 - PARTICIONAMIENTO:
  - Número total de particiones con datos: 1873
  - Puntos por partición (promedio): 26.5

STAGE 2 - DBSCAN LOCAL:
  - Clusters locales identificados: 1
  - Puntos en clusters locales: 2,436
  - Puntos noise/outliers locales: 47,226

STAGE 3 - DETECCIÓN DE CRUCES:
  - Puntos frontera


---

## Referencias y Bibliografía

1. **Paper Original:**
   - He, Y., Tan, H., Luo, W., et al. (2011). "MR-DBSCAN: An Efficient Parallel Density-based Clustering Algorithm using MapReduce"

2. **Algoritmo DBSCAN:**
   - Ester, M., Kriegel, H. P., Sander, J., & Xu, X. (1996). "A density-based algorithm for discovering clusters in large spatial databases"

3. **MapReduce:**
   - Dean, J., & Ghemawat, S. (2004). "MapReduce: Simplified Data Processing on Large Clusters"

4. **Herramientas Utilizadas:**
   - Hadoop: https://hadoop.apache.org/
   - PySpark: https://spark.apache.org/docs/latest/api/python/
   - Scikit-Learn: https://scikit-learn.org/stable/

---